In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
import base64
import requests
from io import BytesIO

# Initialize API
cg = CoinGeckoAPI()

# List of coins and their CoinGecko IDs
coins = {
    'BTC': 'bitcoin',
    'ETH': 'ethereum',
    'ADA': 'cardano',
    'SOL': 'solana',
    'DOT': 'polkadot',
    'AVAX': 'avalanche-2'
}

# Function to fetch OHLC data for the last 365 days (daily granularity)
def fetch_ohlc(coin_id):
    ohlc = cg.get_coin_ohlc_by_id(id=coin_id, vs_currency='usd', days=365)
    df = pd.DataFrame(ohlc, columns=['timestamp', 'open', 'high', 'low', 'close'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    df.drop(columns=['timestamp'], inplace=True)
    return df

# Fetch new OHLC data
ohlc_data = {}
for symbol, coin_id in coins.items():
    df = fetch_ohlc(coin_id)
    df['symbol'] = symbol
    ohlc_data[symbol] = df

# Combine new data into one DataFrame
new_data_df = pd.concat(ohlc_data.values(), ignore_index=True)

# Fetch the current attachment URL from Airtable (replace with your Airtable API calls)
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
airtable_url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"

# Make sure to adjust this query to fetch the latest attachment from Airtable
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json"
}

# Fetch the record that contains the current attachment
response = requests.get(airtable_url, headers=airtable_headers)
response.raise_for_status()
data = response.json()

# Assuming the first record has the attachment we need (you may need to iterate if there are multiple records)
attachment_url = data['records'][0]['fields']['Trending Excel'][0]['url']

# Download the existing file from the attachment URL
existing_file = requests.get(attachment_url)
existing_file.raise_for_status()

# Assume the existing file is Excel format (you can adapt this if it's in CSV)
existing_df = pd.read_excel(BytesIO(existing_file.content))

# Append the new data to the existing data
combined_df = pd.concat([existing_df, new_data_df], ignore_index=True)

# Save the updated data to a new Excel file
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"crypto_ohlc_365days_with_history_{timestamp}.xlsx"
combined_df.to_excel(filename, index=False)

# Upload the updated file to GitHub
GITHUB_REPO = "SagarFieldElevate/DatabaseManagment"
BRANCH = "main"
UPLOAD_PATH = "uploads"
GITHUB_TOKEN = os.getenv("GH_TOKEN")

with open(filename, "rb") as f:
    content = base64.b64encode(f.read()).decode()

upload_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
upload_payload = {
    "message": f"Upload OHLC data with history {timestamp}",
    "content": content,
    "branch": BRANCH
}
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}
upload_resp = requests.put(upload_url, headers=headers, json=upload_payload)
if upload_resp.status_code not in [200, 201]:
    raise Exception(f"❌ GitHub upload failed: {upload_resp.status_code} - {upload_resp.text}")

file_sha = upload_resp.json()["content"]["sha"]
raw_url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{BRANCH}/{UPLOAD_PATH}/{filename}"

# Send the updated file URL to Airtable
airtable_payload = {
    "records": [{
        "fields": {
            "Name": f"OHLC Master Data with History",
            "Database Attachment": [{
                "url": raw_url,
                "filename": filename
            }]
        }
    }]
}

airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=airtable_payload)
if airtable_resp.status_code != 200:
    raise Exception(f"❌ Airtable upload failed: {airtable_resp.status_code} - {airtable_resp.text}")

# Clean up local file
os.remove(filename)
print("🧽 Local file deleted.")